In [2]:
!pip install bs4

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#imported required modules and functions
#Requesting weblink and 

URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1019189119#cite_note-statcan-3'
response = requests.get(URL)
soup = BeautifulSoup(response.text,'html.parser')
#print(soup.prettify())


to get table from html,we need to use find, to find table first and find_all, to find td, wherever it is coming and covert it into a list <br> we gave every column a name in the starting and started appending data into it

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

To convert dict into dataframe

In [5]:
df = pd.DataFrame(table_contents)

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
import geocoder

def get_lati_long(postal_code):
    
    # initialize your variable to None

    lati_long_coord = None
    
    # loop until you get the coordinates
    while(lati_long_coord is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coord = g.latlng
    return lati_long_coord
    
get_lati_long('M5G')

[43.65609000000006, -79.38492999999994]

In [19]:
# Retrieving Postal Code Co-ordinates
postal_codes = df['PostalCode']    
coord = [ get_lati_long(postal_code) for postal_code in postal_codes.tolist() ]

In [20]:
# Adding Columns Latitude & Longitude
df_coord = pd.DataFrame(coord, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coord['Latitude']
df['Longitude'] = df_coord['Longitude']

In [21]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [8]:
df.to_csv('Canada.csv',index=False)

(103, 3)